<a href="https://colab.research.google.com/github/kashindra-mahato/NLP/blob/main/MRKLAgent_(api_keys_required).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MRKLAgent Draft

- Proposal PR on GitHub: https://github.com/deepset-ai/haystack/pull/3925



## Overview

The MRKLAgent class answers queries by choosing between different actions/tools, which are implemented as pipelines.
It uses a large language model (LLM) to generate a thought based on the query, choose an action/tool, and generate the input for the action/tool.
Based on the result returned by an action/tool, the MRKLAgent has two options.
It can either repeat the process of 1) thought, 2) action choice, 3) action input or it stops if it knows the answer.

## Installing Haystack

To start, let's install the latest release of Haystack with `pip`:

In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git@mrkl-pipeline#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git (to revision mrkl-pipeline) to /tmp/pip-install-gmuy360v/farm-haystack_58d712eafaa5472cb680145ba23eebd1
  Resolved https://github.com/deepset-ai/haystack.git to commit c7373b53d73639f498352f83a1e3f03d4b2985f0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-gmuy360v/farm-haystack_58d712eafaa5472cb680145ba23eebd1
  Running command git checkout -b mrkl-pipeline --track origin/mrkl-pipeline
  Switched to a new branch 'mrkl-pipeline'
  Branch 'mrkl-pipeline' set up to track remote branch 'mrkl-pipeline' from 'origin'.


In [ ]:
%%bash

pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Set the logging level to INFO:

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
openai_api_key = ""
serp_api_key = ""

In [ ]:
from haystack import Pipeline
from haystack.nodes import SerpAPIComponent, PythonRuntime, PromptNode, PromptModel
from haystack.nodes.prompt.mrkl_agent import MRKLAgent

## Loading a MRKLAgent from a YAML file

A MRKLAgent can be defined together with its tools (pipelines) in a YAML file. This works just like pipelines in Haystack.

In [ ]:

with open("test.mrkl.haystack-pipeline.yml", "w") as tmp_file:
    tmp_file.write(
        f"""
        version: ignore
        components:
          - name: MRKLAgent
            type: MRKLAgent
            params:
              prompt_node: MRKLAgentPromptNode
              tools: [{{'pipeline_name': 'serpapi_pipeline', 'tool_name': 'Search', 'description': 'useful for when you need to answer questions about current events. You should ask targeted questions'}}, {{'pipeline_name': 'calculator_pipeline', 'tool_name': 'Calculator', 'description': 'useful for when you need to answer questions about math'}}]
          - name: MRKLAgentPromptNode
            type: PromptNode
            params:
              model_name_or_path: DavinciModel
              stop_words: ['Observation:']
          - name: DavinciModel
            type: PromptModel
            params:
              model_name_or_path: 'text-davinci-003'
              api_key: {openai_api_key}
          - name: Serp
            type: SerpAPIComponent
            params:
              api_key: {serp_api_key}
          - name: CalculatorInput
            type: PromptNode
            params:
              model_name_or_path: DavinciModel
              default_prompt_template: CalculatorTemplate
              output_variable: python_runtime_input
          - name: Calculator
            type: PythonRuntime
          - name: CalculatorTemplate
            type: PromptTemplate
            params:
              name: calculator
              prompt_text:  |
                  # Write a simple python function that calculates
                  # $query
                  # Do not print the result; invoke the function and assign the result to final_result variable
                  # Start with import statement

        pipelines:
          - name: mrkl_query_pipeline
            nodes:
              - name: MRKLAgent
                inputs: [Query]
          - name: serpapi_pipeline
            nodes:
              - name: Serp
                inputs: [Query]
          - name: calculator_pipeline
            nodes:
              - name: CalculatorInput
                inputs: [Query]
              - name: Calculator
                inputs: [CalculatorInput]

    """
    )
mrkl_agent = MRKLAgent.load_from_yaml(
    "test.mrkl.haystack-pipeline.yml", pipeline_name="mrkl_query_pipeline"
)


INFO:haystack.pipelines.config:Missing definition for node of type MRKLAgent. Looking into local classes...
INFO:haystack.nodes._json_schema:Added definition for MRKLAgent
INFO:haystack.pipelines.config:Missing definition for node of type MRKLAgent. Looking into local classes...
INFO:haystack.nodes._json_schema:Added definition for MRKLAgent
INFO:haystack.pipelines.config:Missing definition for node of type MRKLAgent. Looking into local classes...
INFO:haystack.nodes._json_schema:Added definition for MRKLAgent


In [ ]:
# result = mrkl_agent.run(query="What is 2 to the power of 3?")
# assert result[1] == "2 to the power of 3 is 8."

result = mrkl_agent.run(query="Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")
print(result[0])
print(result[1])

https://serpapi.com/search
https://serpapi.com/search
Answer the following questions as best as you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events. You should ask targeted questions
Calculator: useful for when you need to answer questions about math

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: $query
Thought: I need to find information about Olivia Wilde's boyfriend, as well as do a calculation.
Action: Search
Action Input: Olivia Wilde boyfriend
Observation: Jason Sudeikis
Thought: I now need to find out his age
A

The expected answer is something like: `"Jason Sudeikis, Olivia Wilde's boyfriend, is 47 years old and his age raised to the 0.23 power is 2.4242784855673896."`

## Programmatically creating a MRKLAgent

1. Create `PromptNode` with davinci `PromptModel`
2. Create a search pipeline containing a `SerpAPIComponent`
3. Describe what the search pipeline can do as a tool
4. Initialize `MRKLAgent` with `PromptNode`, search pipeline within `tool_map` and description of the tool in `tools`

In [ ]:
prompt_model = PromptModel(model_name_or_path="text-davinci-003", api_key=openai_api_key)
prompt_node = PromptNode(model_name_or_path=prompt_model, stop_words=["Observation:"])

search = SerpAPIComponent(api_key=serp_api_key)
search_pipeline = Pipeline()
search_pipeline.add_node(component=search, name="Serp", inputs=["Query"])

tools = [
    {
        "pipeline_name": "serpapi_pipeline",
        "tool_name": "Search",
        "description": "useful for when you need to answer questions about current events. You should ask targeted questions",
    }
]
tool_map = {"Search": search_pipeline}
mrkl_agent = MRKLAgent(prompt_node=prompt_node, tools=tools, tool_map=tool_map)

In [ ]:
result = mrkl_agent.run(query="What is 2 to the power of 3?")
# This MRKLAgent has no access to a calculator but Search can also solve these simple calculator questions.
print(result[0])
print(result[1])

https://serpapi.com/search
Answer the following questions as best as you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events. You should ask targeted questions

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: $query
Thought: How can I calculate this?
Action: Search
Action Input: "2 to the power of 3"
Observation: Answer: 2 raised to the third power is equal to 23 = 8. ... Explanation: 2 to the 3rd power can be written as 23 = 2 × 2 × 2, as 2 is multiplied by itself 3 times ...
Thought: I now know the final answer
Final Answer: 2 to the

## About us

This [Haystack](https://github.com/deepset-ai/haystack/) notebook was made with love by [deepset](https://deepset.ai/) in Berlin, Germany

We bring NLP to the industry via open source!  
Our focus: Industry specific language models & large scale QA systems.  
  
Some of our other work:
- [German BERT](https://deepset.ai/german-bert)
- [GermanQuAD and GermanDPR](https://deepset.ai/germanquad)

Get in touch:
[Twitter](https://twitter.com/deepset_ai) | [LinkedIn](https://www.linkedin.com/company/deepset-ai/) | [Discord](https://haystack.deepset.ai/community) | [GitHub Discussions](https://github.com/deepset-ai/haystack/discussions) | [Website](https://deepset.ai)

By the way: [we're hiring!](https://www.deepset.ai/jobs)
